In [ ]:
## the following code was run in Google Colaboratory (so all group members could work on the code simultaneousy and remotely)

# Import relevant packages
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import datetime

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix, auc, roc_curve, plot_roc_curve, plot_precision_recall_curve
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ECO 416 FINAL PROJECT/Datasets/VEHICLE_LOAN_DEFAULT/train.csv')

from google.colab import drive
drive.mount('/content/drive')

data = data.drop(columns=['UniqueID'])

def str_to_months(foo):
  foo = foo.replace("mon", "yrs")
  foo = foo.replace("yrs",'')
  temparr = foo.split(" ")
  return int(temparr[0])*12 + int(temparr[1])

data['CREDIT.HISTORY.LENGTH'] = data.apply(lambda row: str_to_months(row['CREDIT.HISTORY.LENGTH']), axis = 1)
data['AVERAGE.ACCT.AGE'] = data.apply(lambda row: str_to_months(row['AVERAGE.ACCT.AGE']), axis = 1)
data['Date.of.Birth'] = data.apply(lambda row: datetime.datetime.strptime(row['Date.of.Birth'],'%d-%m-%y'), axis = 1)
min_date = min(data['Date.of.Birth'])
data['Date.of.Birth'] = data.apply(lambda row: (row['Date.of.Birth'] - min_date)/datetime.timedelta(days = 1), axis = 1)
data['DisbursalDate'] = data.apply(lambda row: datetime.datetime.strptime(row['DisbursalDate'],'%d-%m-%y'), axis = 1)
min_date = min(data['DisbursalDate'])
data['DisbursalDate'] = data.apply(lambda row: (row['DisbursalDate'] - min_date)/datetime.timedelta(days = 1), axis = 1)
X = data.drop('loan_default', axis=1).drop('Employee_code_ID', axis=1).drop('Current_pincode_ID',axis=1).drop('branch_id',axis=1).drop('State_ID',axis=1).drop('supplier_id', axis=1)
y = data['loan_default']
X = pd.get_dummies(X, columns=['Employment.Type', 'PERFORM_CNS.SCORE.DESCRIPTION','manufacturer_id'])
var_num = ['disbursed_amount','asset_cost','ltv',
           'Date.of.Birth','DisbursalDate','MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag','VoterID_flag',
           'Driving_flag', 'Passport_flag','PERFORM_CNS.SCORE','PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS',
           'PRI.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.NO.OF.ACCTS','SEC.ACTIVE.ACCTS','SEC.OVERDUE.ACCTS',
           'SEC.CURRENT.BALANCE','SEC.SANCTIONED.AMOUNT','SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
           'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH','NO.OF_INQUIRIES']
X_numt = X[var_num]
X_catt = X.drop(columns=var_num, axis=1)

scaler = StandardScaler()
X_num_sct = pd.DataFrame(scaler.fit_transform(X_numt))
X_num_sct.columns = X_numt.columns
X_sct = X_num_sct.join(X_catt)

clf_rf1 = RandomForestClassifier().fit(X_sct, y)
featimparr = np.hstack((np.array(clf_rf1.feature_names_in_).reshape(-1,1), np.array(clf_rf1.feature_importances_).reshape(-1,1)))
featimpdf = pd.DataFrame(featimparr, columns = ['Feature', 'Importance'])

def plot_feature_importances(df):
  # Sort features according to importance
    df = df.sort_values('Importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['Importance'] / df['Importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (14, 10))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['Feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

plot_feature_importances(featimpdf)

plt.boxplot(X['Date.of.Birth'][y == 0])
plt.boxplot(X['Date.of.Birth'][y == 1], positions=[2])
plt.xticks(ticks=[1,2],labels=[0,1])
plt.title('DOB')
plt.show()

plt.boxplot(X['ltv'][y == 0])
plt.boxplot(X['ltv'][y == 1], positions=[2])
plt.xticks(ticks=[1,2],labels=[0,1])
plt.title('ltv')
plt.show()

plt.boxplot(X['DisbursalDate'][y == 0])
plt.boxplot(X['DisbursalDate'][y == 1], positions=[2])
plt.xticks(ticks=[1,2],labels=[0,1])
plt.title('Disbursal Date')
plt.show()

In [ ]:
data2 = pd.read_csv('/content/drive/MyDrive/ECO 416 FINAL PROJECT/Datasets/UCI_Credit_Card.csv')

data2 = data2.drop(columns=['ID'])
X = data2.drop('default.payment.next.month', axis=1)
y = data2['default.payment.next.month']
X = pd.get_dummies(X, columns=['MARRIAGE'])

var_num = ['LIMIT_BAL','SEX','EDUCATION','AGE','PAY_0','PAY_2', 'PAY_3',
           'PAY_4','PAY_5','PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3','BILL_AMT4',
           'BILL_AMT5', 'BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5',
           'PAY_AMT6']
X_numt = X[var_num]
X_catt = X.drop(columns=var_num, axis=1)

scaler = StandardScaler()
X_num_sct = pd.DataFrame(scaler.fit_transform(X_numt))
X_num_sct.columns = X_numt.columns
X_sct = X_num_sct.join(X_catt)

clf_rf = RandomForestClassifier().fit(X_sct, y)
# How a tree looks like 
plt.figure(figsize=(10,10))

plot_tree(DecisionTreeClassifier(max_depth=2).fit(X_sct, y))
plt.show()

featimparr = np.hstack((np.array(clf_rf.feature_names_in_).reshape(-1,1), np.array(clf_rf.feature_importances_).reshape(-1,1)))
featimpdf = pd.DataFrame(featimparr, columns = ['Feature', 'Importance'])

plot_feature_importances(featimpdf)

plt.boxplot(X['LI'][y == 0])
plt.boxplot(X['BILL_AMT1'][y == 1], positions=[2])
plt.xticks(ticks=[1,2],labels=[0,1])
plt.title('BILL_AMT1')
plt.show()

plt.boxplot(X['PAY_0'][y == 0])
plt.boxplot(X['PAY_0'][y == 1], positions=[2])
plt.xticks(ticks=[1,2],labels=[0,1])
plt.title('Default last month')
plt.show()

In [ ]:
data3 = pd.read_csv('/content/drive/MyDrive/ECO 416 FINAL PROJECT/Datasets/home-credit-default-risk/application_train.csv')
data3.dropna(inplace = True)
data3.reset_index(drop=True, inplace=True)

data3=data3.drop('SK_ID_CURR',axis=1).drop('EXT_SOURCE_3',axis=1).drop('EXT_SOURCE_2',axis=1).drop('EXT_SOURCE_1',axis=1)
X = data3.drop('TARGET', axis=1)
y = data3['TARGET']
a = ['NAME_CONTRACT_TYPE', 'CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE',
                               'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
                               'OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE',
                               'WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']
X_numt = X.drop(columns=a,axis=1)
X = pd.get_dummies(X, columns=a)
X_catt = X.drop(columns=X_numt.columns, axis=1)
scaler = StandardScaler()
X_num_sct = pd.DataFrame(scaler.fit_transform(X_numt))
X_num_sct.columns = X_numt.columns
# X_sct = X_num_sct.join(X_catt)
X_sct = pd.merge(X_num_sct, X_catt, left_index=True, right_index=True)
X_catt.head()

numtcolarr = np.array([i for i in data3.columns])
val1arr = np.array([data3.iloc[0][i] for i in data3.columns])
val2arr = np.array([data3.iloc[1][i] for i in data3.columns])
final_arr = np.hstack((colarr.reshape(-1,1), val1arr.reshape(-1,1), val2arr.reshape(-1,1)))
coldf = pd.DataFrame(final_arr, columns = ['column_name', '1st_value','2nd_value'])
coldf.head()
coldf.to_csv('/content/drive/MyDrive/ECO 416 FINAL PROJECT/columns.csv')

clf_rf2 = RandomForestClassifier().fit(X_sct, y)

featimparr = np.hstack((np.array(clf_rf2.feature_names_in_).reshape(-1,1), np.array(clf_rf2.feature_importances_).reshape(-1,1)))
featimpdf = pd.DataFrame(featimparr, columns = ['Feature', 'Importance'])
plot_feature_importances(featimpdf)

plt.boxplot((X['DAYS_BIRTH'] * -1)[y == 0])
plt.boxplot((X['DAYS_BIRTH'] * -1)[y == 1], positions=[2])
plt.xticks(ticks=[1,2],labels=[0,1])
plt.show()